In [4]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import pickle


ModuleNotFoundError: No module named 'surprise'

In [5]:
!pip install scikit-surprise --prefer-binary


  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build scikit-surprise


  error: subprocess-exited-with-error
  
  × Building wheel for scikit-surprise (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [155 lines of output]
      C:\Users\Najwa\AppData\Local\Temp\pip-build-env-z1bgpxbn\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ****************************************

In [1]:
import pandas as pd
import numpy as np


In [3]:
## import data
#book=pd.read_csv('../data/raw/book.csv')

book_path='../data/processed/books_cleaned.csv'
rating_path='../data/processed/ratings_cleaned.csv'
user_path='../data/raw/Users.csv'
books = pd.read_csv(book_path, delimiter=",")
ratings = pd.read_csv(rating_path, delimiter=",")
users = pd.read_csv(user_path, delimiter=",")



C:\Users\Najwa\AppData\Local\Temp\ipykernel_16072\2215916709.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(book_path, delimiter=",")


In [14]:
# Bersihkan nama kolom
ratings.columns = ratings.columns.str.strip()
ratings = ratings[['User-ID', 'ISBN', 'Book-Rating']]
ratings.columns = ['user_id', 'isbn', 'rating']
ratings.info()

KeyError: "None of [Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')] are in the [columns]"

In [15]:
# Gunakan hanya rating eksplisit (>0)
ratings = ratings[ratings['rating'] > 0]
len(ratings)

433671

In [16]:
# ===== STEP 2: Siapkan Dataset untuk Surprise =====
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings[['user_id', 'isbn', 'rating']], reader)

# ===== STEP 3: Train-Test Split =====
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [17]:
model = SVD()
model.fit(trainset)

In [18]:
# ===== STEP 5: Evaluasi =====
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

# ===== STEP 6: Simpan Model =====
with open("svd_model.pkl", "wb") as f:
    pickle.dump(model, f)

RMSE: 1.6394
RMSE: 1.639420461704921


In [3]:
import numpy as np
import pandas as pd

In [ ]:

import pickle

# Load model dan dataset
with open("svd_model.pkl", "rb") as f:
    model = pickle.load(f)

ratings = pd.read_csv(rating_path, delimiter=",")
books = pd.read_csv(book_path, delimiter=",")

ratings.columns = ratings.columns.str.strip()
ratings = ratings[['User-ID', 'ISBN', 'Book-Rating']]
ratings.columns = ['user_id', 'isbn', 'rating']
ratings = ratings[ratings['rating'] > 0]

# Ambil list buku yang belum dirating user
def get_top_n_recommendations(user_id, n=5):
    all_books = ratings['isbn'].unique()
    rated_books = ratings[ratings['user_id'] == user_id]['isbn']
    books_to_predict = [isbn for isbn in all_books if isbn not in rated_books.values]

    predictions = [(isbn, model.predict(user_id, isbn).est) for isbn in books_to_predict]
    top_n = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

    # Gabungkan dengan judul
    books.columns = books.columns.str.strip()
    result = []
    for isbn, pred_rating in top_n:
        title = books[books['ISBN'] == isbn]['Book-Title'].values
        title = title[0] if len(title) > 0 else 'Unknown'
        result.append((isbn, title, round(pred_rating, 2)))

    return result

# Contoh: rekomendasi untuk user ID 276729
print(get_top_n_recommendations(276729))


ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: numpy._core.multiarray failed to import